In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import time

class HealthConditionDetector:
    def __init__(self, data_path):
        self.df = pd.read_csv(r"C:\Users\vpran\OneDrive\Desktop\rotatechX\simulated_health_data.csv")
        
        # Extract BP components
        self.df[['Systolic_BP', 'Diastolic_BP']] = self.df['Blood Pressure'].str.extract(r'(\d+)/(\d+)').astype(float)
        self.df = self.df.drop(columns=['Blood Pressure'])
        
        # Encode health conditions
        self.label_encoder = LabelEncoder()
        self.df['Health Condition'] = self.label_encoder.fit_transform(self.df['Health Condition'])
        
        # Prepare features and target
        self.X = self.df.drop(columns=['Health Condition'])
        self.y = self.df['Health Condition']
        
        # Scale features
        self.scaler = StandardScaler()
        self.X_scaled = self.scaler.fit_transform(self.X)
        
        # Train model
        self.model = self.train_model()
    
    def train_model(self):
        X_train, X_test, y_train, y_test = train_test_split(
            self.X_scaled, self.y, test_size=0.2, random_state=42
        )
        model = DecisionTreeClassifier(max_depth=7, random_state=42)
        model.fit(X_train, y_train)
        return model

    def assess_health(self, vitals_dict):
        user_df = pd.DataFrame(vitals_dict, index=[0])
        user_scaled = self.scaler.transform(user_df)
        pred_encoded = self.model.predict(user_scaled)[0]
        return self.label_encoder.inverse_transform([pred_encoded])[0]

def stream_and_monitor(csv_path):
    detector = HealthConditionDetector(r"C:\Users\vpran\OneDrive\Desktop\rotatechX\simulated_health_data.csv")
    raw_df = pd.read_csv(csv_path)

    for index, row in raw_df.iterrows():
        systolic, diastolic = map(float, row['Blood Pressure'].split('/'))
        
        user_data = {
            'Heart Rate': row['Heart Rate'],
            'SpO2': row['SpO2'],
            'Steps': row['Steps'],
            'Sleep': row['Sleep'],
            'Calories Burnt': row['Calories Burnt'],
            'Systolic_BP': systolic,
            'Diastolic_BP': diastolic
        }

        # Display the current vitals
        print(f"--- Reading {index + 1} ---")
        print(f"Heart Rate    : {user_data['Heart Rate']} bpm")
        print(f"SpO2          : {user_data['SpO2']}%")
        print(f"Steps         : {user_data['Steps']}")
        print(f"Sleep         : {user_data['Sleep']} hrs")
        print(f"Calories Burnt: {user_data['Calories Burnt']} kcal")
        print(f"Blood Pressure: {systolic}/{diastolic} mmHg")

        result = detector.assess_health(user_data)
        print(f"\n🩺 Condition Detected: {result}")

        if result == "Emergency":
            print("🚨 EMERGENCY ALERT! Immediate attention required!")
            print("📩 Alert msg sent to +91 7358494750")
        elif result == "Unhealthy":
            print("⚠️ Health check recommended.")
        else:
            print("✅ All vitals normal.")

        print("\n" + "-"*40 + "\n")  # Line break between entries
        time.sleep(0.5)

if __name__ == "__main__":
    stream_and_monitor(r"C:\Users\vpran\OneDrive\Desktop\rotatechX\simulated_health_data.csv")


--- Reading 1 ---
Heart Rate    : 74.05748366 bpm
SpO2          : 98.25144931%
Steps         : 9374.591003
Sleep         : 7.693121875 hrs
Calories Burnt: 413.6332501 kcal
Blood Pressure: 120.0/77.0 mmHg

🩺 Condition Detected: Healthy
✅ All vitals normal.

----------------------------------------

--- Reading 2 ---
Heart Rate    : 103.6770107 bpm
SpO2          : 93.27832585%
Steps         : 2982.590391
Sleep         : 4.978162049 hrs
Calories Burnt: 181.150332 kcal
Blood Pressure: 150.0/96.0 mmHg

🩺 Condition Detected: Unhealthy
⚠️ Health check recommended.

----------------------------------------

--- Reading 3 ---
Heart Rate    : 113.3191764 bpm
SpO2          : 93.31318562%
Steps         : 3566.650395
Sleep         : 5.753348379 hrs
Calories Burnt: 102.1762602 kcal
Blood Pressure: 149.0/94.0 mmHg

🩺 Condition Detected: Unhealthy
⚠️ Health check recommended.

----------------------------------------

--- Reading 4 ---
Heart Rate    : 37.30596174 bpm
SpO2          : 86.50609111%
Steps